## Useful references
- OSMnx docs: https://osmnx.readthedocs.io/en/stable/
- OSMnx examples: https://github.com/gboeing/osmnx-examples
- Shapely manual: https://shapely.readthedocs.io/en/stable/manual.html
- Folium docs: https://python-visualization.github.io/folium/index.html
- Available markers icons: https://www.w3schools.com/bootstrap/bootstrap_ref_comp_glyphs.asp
- Directed angle between vectors: https://it.mathworks.com/matlabcentral/answers/180131-how-can-i-find-the-angle-between-two-vectors-including-directional-information

In [ ]:
import networkx as nx
import osmnx as ox
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import LineString
from math import atan2, degrees

ox.settings.log_console = True
data_folder = "../data/"

In [ ]:
def marker_from_row(row: pd.Series, m: folium.Map, icon: str = "info-sign", color: str = "blue"):
    """
    row: Series representing the POI to plot
    m: folium map object
    icon: icon name
    color: marker color
    """
    long, lat = row.geometry.x, row.geometry.y
    folium.Marker(
        location=[lat, long],
        popup=row.to_string(),
        icon=folium.Icon(icon=icon),
        color=color
    ).add_to(m)


def plot_pois(m: folium.Map, pois: gpd.GeoDataFrame, icon: str = "info-sign", color: str = "blue") -> folium.Map:
    """
    m: folium map object
    pois: POIs GeoDataFrame
    icon: icon name for markers
    color: marker color
    """
    pois.apply(lambda row: marker_from_row(row, m, icon, color), axis=1)
    return m

In [ ]:
place = 'Mantova, Italia'

fname = place.split(',')[0].lower()

boundaries_gdf = ox.geocode_to_gdf(place)
boundaries_poly = boundaries_gdf.unary_union

In [ ]:
graph = ox.graph_from_polygon(boundaries_poly, network_type='drive')
graph = ox.add_edge_bearings(graph)
graph = ox.add_edge_speeds(graph)
graph = ox.add_edge_travel_times(graph)
ox.save_graphml(graph, f"{data_folder}{fname}_graph.graphml")

traffic_lights = ox.geometries_from_polygon(boundaries_poly, tags={'highway': 'traffic_signals'})
traffic_lights.to_file(f"{data_folder}{fname}_traffic-lights.geojson")

In [ ]:
graph = ox.load_graphml(f"{data_folder}{fname}_graph.graphml")
traffic_lights = gpd.read_file(f"{data_folder}{fname}_traffic-lights.geojson")

In [ ]:
nodes, edges = ox.graph_to_gdfs(graph)
nodes['df_index'] = nodes.index
edges['df_index'] = edges.index

m = edges.explore(tiles="CartoDB positron", popup=True, tooltip=False, color="grey")
m = nodes.explore(m=m, popup=True, tooltip=False, color="grey")
m = traffic_lights.explore(m=m, color="cyan")
# plot_pois(m, traffic_lights, "option-vertical")
m

In [ ]:
# Origin and destination nodes
orig_node = 1818853340
dest_node = 1313376786
# Cost to minimize (pick from edge attributes)
weight = 'length'

path = ox.shortest_path(graph, orig_node, dest_node, weight, cpus=None)
marker_from_row(row=nodes.loc[orig_node], m=m, icon="flag", color="green")
marker_from_row(row=nodes.loc[dest_node], m=m, icon="screenshot", color="red")

node_pairs = zip(path[:-1], path[1:])
uvk = ((u, v, min(graph[u][v].items(), key=lambda k: k[1]["length"])[0]) for u, v in node_pairs)
path_edges = ox.graph_to_gdfs(graph.subgraph(path), nodes=False).loc[uvk]

path_edges["df_index"] = path_edges.index
m = path_edges.explore(m=m, popup=True, tooltip=False, color="blue")
m

In [ ]:
left_turns = 0
iterator = path_edges.itertuples()
prev = next(iterator)
prev_end = LineString(prev.geometry.coords[-2:])

for curr in iterator:
    u, v, _ = curr.Index
    
    if nodes.loc[u, 'street_count'] < 3:
        continue
        
    curr_start = LineString(curr.geometry.coords[:2])
        
    folium.PolyLine(locations=[coord[::-1] for coord in prev_end.coords], color="red").add_to(m)
    folium.PolyLine(locations=[coord[::-1] for coord in curr_start.coords], color="green").add_to(m)
    
    # angolo tra prev e curr compreso tra 45 e 135
    x1 = prev_end.coords[1][0] - prev_end.coords[0][0]
    y1 = prev_end.coords[1][1] - prev_end.coords[0][1]
    x2 = curr_start.coords[1][0] - curr_start.coords[0][0]
    y2 = curr_start.coords[1][1] - curr_start.coords[0][1]
    theta = degrees(atan2(x1 * y2 - y1 * x2, x1 * x2 + y1 * y2))
    
    print(f"Angle between edge {prev.osmid} and {curr.osmid}: {theta} degrees")
    
    if 45 <= theta <= 135:
        left_turns += 1
        row = nodes.loc[u]
        marker_from_row(row=row, m=m, icon="arrow-left")
    
    prev = curr
    prev_end = LineString(prev.geometry.coords[-2:])  
    
print(f"Number of left turns: {left_turns}")
m  

In [ ]:
n_paths = 100
paths = ox.k_shortest_paths(graph, orig=orig_node, dest=dest_node, k=n_paths, weight="travel_time")
paths = list(paths)
colors = ox.plot.get_colors(n=n_paths, cmap="rainbow", return_hex=True)
for path, color in zip(paths, colors):
    m = ox.plot_route_folium(graph, route=path, route_map=m, color=color)